In [1]:
import pandas as pd
import numpy as np
import tensorflow
import keras

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('/Users/aaronmichaelbroderick/Desktop/Data Science/Data/devanagari.csv')

In [3]:
Y = data['character']

columns = list(data)
columns.remove('character')
X = data[columns]

In [4]:
# Change strings into numerical to work well with Keras

letters = list(set(Y))
Y_test = list(Y)

for i in range(len(letters)):
    for j in range(len(Y_test)):
        if letters[i] == Y_test[j]:
            Y_test[j] = i

In [5]:
results = pd.DataFrame()
results['cat'] = Y
results['num'] = Y_test

Y = results.num

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)

In [7]:
# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 46)
y_test = keras.utils.to_categorical(y_test, 46)

73600 train samples
18400 test samples


In [8]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(1024,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(46, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 46)                2990      
Total params: 72,750
Trainable params: 72,750
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(x_train, y_train,
                    batch_size=500,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 73600 samples, validate on 18400 samples
Epoch 1/10
73600/73600 [==============================] - 3s 38us/step - loss: 2.3547 - acc: 0.3762 - val_loss: 1.4812 - val_acc: 0.6098
Epoch 2/10
73600/73600 [==============================] - 3s 36us/step - loss: 1.4649 - acc: 0.5914 - val_loss: 1.1298 - val_acc: 0.6960
Epoch 3/10
73600/73600 [==============================] - 3s 37us/step - loss: 1.2201 - acc: 0.6536 - val_loss: 0.9487 - val_acc: 0.7424
Epoch 4/10
73600/73600 [==============================] - 3s 36us/step - loss: 1.0786 - acc: 0.6899 - val_loss: 0.8612 - val_acc: 0.7588
Epoch 5/10
73600/73600 [==============================] - 3s 36us/step - loss: 0.9668 - acc: 0.7191 - val_loss: 0.7491 - val_acc: 0.7904
Epoch 6/10
73600/73600 [==============================] - 3s 37us/step - loss: 0.8945 - acc: 0.7395 - val_loss: 0.7021 - val_acc: 0.7995
Epoch 7/10
73600/73600 [==============================] - 3s 37us/step - loss: 0.8404 - acc: 0.7543 - val_loss: 0.6770 - val_acc